# Projeto Spark

Entrega: 31 de maio de 2023

## Introdução

Neste projeto vamos construir um classificador Naive-Bayes para determinar o sentimento de um comentário.

## Grupos

O projeto pode ser individual ou em duplas. Criem os grupos em https://classroom.github.com/a/EdbpYF9x

## Instalando o ambiente

O jeito mais simples de começar a trabalhar com Spark é instalar um container com tudo pronto! No site https://hub.docker.com/r/jupyter/pyspark-notebook vemos uma imagem Docker que já vem com `pyspark` e `jupyter lab`. Instale a imagem com o comando:

```bash
docker pull jupyter/pyspark-notebook
```

Vamos iniciar o ambiente de trabalho com o comando `docker run`. Para isso precisamos tomar alguns cuidados:

1) Temos que mapear nosso diretorio local de trabalho para um diretório interno do container, de modo que alterações feitas dentro do container (nesta pasta escolhida) sejam gravadas no nosso diretorio local. No container temos um usuário padrão com *username* `jovyan`. No *homedir* desse usuario temos uma pasta vazia `work`, que vai servir como local de mapeamento do nosso diretorio local de trabalho. Podemos então fazer esse mapeamendo com a opção `-v` do comando `docker run` da seguinte forma:

```bash
-v <diretorio>:/home/jovyan/work
```

onde `<diretorio>` representa seu diretorio local de trabalho.

2) Para acessar o `jupyter notebook` e o *dashboard* do Spark a partir do nosso *browser* favorito temos que abrir algumas portas do container com a opção `-p`. As portas são `8888` (para o próprio `jupyter notebook`) e `4040` (para o *dashboard* do Spark). Ou seja, adicionaremos às opções do `docker run`o seguinte:

```bash
-p 8888:8888 -p 4040:4040
```

Desta forma, ao acessar `localhost:8888` na nossa máquina, estaremos acessando o servidor Jupyter na porta 8888 interna do container.

3) Vamos iniciar o container no modo interativo, e vamos especificar que o container deve ser encerrado ao fechar o servidor Jupyter. Faremos isso com as opções `-it` e `-rm`

Portanto, o comando completo que eu uso na minha máquina Linux para iniciar o container é:

```bash
docker run \
    -it \
    --rm \
    -p 8888:8888 \
    -p 4040:4040 \
    -v "`pwd`":/home/jovyan/work \
    jupyter/pyspark-notebook
```

Para facilitar a vida eu coloco esse comando em um arquivo `inicia.sh`. Engenheiros, façam do jeito que preferirem!

Agora abra esse notebook lá no container!


## Iniciando o Spark

Vamos iniciar o ambiente Spark. Para isso vamos:

1) Criar um objeto de configuração do ambiente Spark. Nossa configuração será simples: vamos especificar que o nome da nossa aplicação Spark é "Minha aplicação", e que o *master node* é a máquina local, usando todos os *cores* disponíveis. Aplicações reais de Spark são configuradas de modo ligeiramente diferente: ao especificar o *master node* passamos uma URL real, com o endereço do nó gerente do *cluster* Spark.

2) Vamos criar um objeto do tipo `SparkContext` com essa configuração

In [93]:
sc.stop()

In [1]:
import pyspark
conf = pyspark.SparkConf()
conf.setAppName('Minha aplicação')
conf.setMaster('local[*]')

sc = pyspark.SparkContext(conf=conf)

O `SparkContext` é a nossa porta de entrada para o cluster Spark, ele será a raiz de todas as nossas operações com o Spark.

In [2]:
sc

<SparkContext master=local[*] appName=Minha aplicação>

O link acima provavelmente não funcionará porque ele se refere à porta 4040 interna do container (portanto a URL está com endereço interno). Porém fizemos o mapeamento da porta 4040 interna para a porta 4040 externa, logo você pode acessar o *dashboard* do Spark no endereço http://localhost:4040

<center><img src="./spark_dashboard.png" width=800/></center>

## Lendo os dados

Vamos começar lendo o arquivo de reviews e gravando o resultado em formato pickle, mais amigável.

In [3]:
def parse_line(line):
    parts = line[1:-1].split('","')
    sentiment = int(parts[0])
    title = parts[1].replace('""', '"')
    body = parts[2].replace('""', '"')
    return (sentiment, title, body)

rdd = sc.textFile('train.csv').map(parse_line)

In [4]:
rdd.count()

3600000

In [5]:
rdd.take(1)

[(2,
  'Stuning even for the non-gamer',
  'This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^')]

Agora vamos gravar no formato pickle, para facilitar os trabalhos futuros. Após gravar o arquivo, não mais rode as células desta primeira etapa!

In [6]:
rdd.saveAsPickleFile('reviews.pickle')

Py4JJavaError: An error occurred while calling o53.saveAsObjectFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/home/jovyan/work/reviews.pickle already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:299)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1091)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:405)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1089)
	at org.apache.spark.rdd.PairRDDFunctions

## Um classificador Naive-Bayes

Vamos ler o arquivo pickle gravado anteriormente:

In [7]:
rdd = sc.pickleFile('reviews.pickle')

In [8]:
rdd.count()

3600000

In [9]:
rdd.take(1)

[(2,
  'Does what they say',
  "This lift does a great job. I had it installed in about one hour by myself. But we have a loft-type ceiling. My wife loved it so much we are buying two or three more. If I get her help this time, it probably won't take as long.")]

In [10]:
x = rdd.take(1)
len(x[0])

3

Agora, complete as tarefas em sequencia para construir o classificador Naive-Bayes:

### Fase 1

#### Tarefa

Construa uma função que recebe um RDD no formato do RDD original e retorna um RDD no qual cada item é um par (palavra, contagem).

In [11]:
import re

def remover_pontuacao(texto):
    padrao = r"[^\w\s]"    
    texto_sem_pontuacao = re.sub(padrao, "", texto)
    return texto_sem_pontuacao


def contar_palavras(rdd):
    rdd = rdd.flatMap(lambda x: (x[1] + ' ' + x[2]).strip().lower().split())
    
    rdd = rdd.map(remover_pontuacao)
    
    rdd = rdd.filter(lambda x: x != "")


    pares = rdd.map(lambda palavra: (palavra, 1))
    
    contagens = pares.reduceByKey(lambda x, y: x + y)
    
    return contagens

contagens = contar_palavras(rdd)
contagens.take(1)

[('4', 115578)]

In [12]:
contagens.take(20)

[('4', 115578),
 ('im', 343336),
 ('bikes', 3142),
 ('came', 126367),
 ('especially', 90730),
 ('like', 1016054),
 ('out', 807325),
 ('clearance', 1522),
 ('start', 83620),
 ('item', 124508),
 ('installation', 10916),
 ('holds', 21137),
 ('common', 23424),
 ('complain', 8697),
 ('family', 91251),
 ('attaching', 1323),
 ('book', 1926132),
 ('willing', 11392),
 ('manager', 4001),
 ('references', 11961)]

In [13]:
rdd.take(1)

[(2,
  'Does what they say',
  "This lift does a great job. I had it installed in about one hour by myself. But we have a loft-type ceiling. My wife loved it so much we are buying two or three more. If I get her help this time, it probably won't take as long.")]

#### Tarefa

Construa uma função que recebe o RDD (palavra, contagem) construido anteriormente e retorna um RDD no qual cada item é um par (palavra, $\log_{10}\left(c \, / \, T\right)$), onde $c$ é a contagem daquela palavra e $T$ é a soma das contagens de palavra.

In [14]:
import math

def calcular_log_probabilidades(rdd):
    T = contagens.map(lambda x: x[1]).reduce(lambda x, y: x + y)
    
    rdd = rdd.map(lambda x: (x[0], math.log(x[1]/T, 10)))
    
    return rdd
                  

contagensLog = calcular_log_probabilidades(contagens)
contagensLog.take(1)

[('4', -3.3864777882880177)]

#### Tarefa

Separe o RDD original em dois RDDs: o dos reviews positivos e o dos negativos. Em seguida, use as funções anteriores para construir RDDs que contem os pares (palavra, $\log_{10}\left(c \, / \, T\right)$)

In [15]:
rdd_positivo = rdd.filter(lambda x: x[0] == 2)
rdd_negativo = rdd.filter(lambda x: x[0] == 1)

rdd_positivo = contar_palavras(rdd_positivo)
rdd_negativo = contar_palavras(rdd_negativo)

rdd_positivo = calcular_log_probabilidades(rdd_positivo)
rdd_negativo = calcular_log_probabilidades(rdd_negativo)

### Tarefa

Use o `.fullOuterJoin()` dos RDDs para construir um RDD unificado, no qual cada item é da forma (palavra, log_prob_positivo, log_prob_negativo). "Baixe" esse resultado final usando `.collect()`.

In [16]:
rdd_positivo.take(10)

[('bikes', -5.213319816190182),
 ('came', -3.666658232102368),
 ('4', -3.6793704103783034),
 ('out', -2.8930396053141987),
 ('clearance', -5.680714862060204),
 ('start', -3.8015007392502964),
 ('like', -2.7696507664476715),
 ('item', -3.913110256469499),
 ('installation', -4.7372928208467435),
 ('holds', -4.3019239946091625)]

In [17]:
rdd_negativo.take(10)

[('4', -3.695800543554358),
 ('im', -3.167463220950031),
 ('especially', -3.936175330828775),
 ('like', -2.7187709290231235),
 ('out', -2.7987385306226753),
 ('clearance', -5.4785413524353),
 ('came', -3.631549689342251),
 ('book', -2.48269742386178),
 ('willing', -4.632714018409357),
 ('manager', -5.112093423557543)]

In [22]:
rdd_joined = rdd_positivo.fullOuterJoin(rdd_negativo)
rdd_joined.take(10)

ConnectionRefusedError: [Errno 111] Connection refused

#### Tarefa

Para uma dada string, determine se ela é um review positivo ou negativo usando os RDDs acima. Lembre-se de como funciona o classificador Naive-Bayes: http://stanford.edu/~jurafsky/slp3/slides/7_NB.pdf, consulte tambem suas notas de aula de Ciência dos Dados!

In [21]:
def NaiveBayes(review, wordsProb):    
    log_prob_positivo_total = 0
    log_prob_negativo_total = 0
    
    for word in review.split():
        if word in wordsProb.map(lambda x: x[0]):
            log_prob_positivo, log_prob_negativo = wordsProb.filter(lambda x: x[0] == word).first()[1]
            if log_prob_positivo is not None:
                log_prob_positivo_total += log_prob_positivo
            if log_prob_negativo is not None:
                log_prob_negativo_total += log_prob_negativo
    
    if log_prob_positivo_total > log_prob_negativo_total:
        return 2
    return 1


# Adicionando "coluna" para verificar teste do classificador
# classification=[]
# rdd_result = rdd
# reviews = rdd_result.map(lambda x: x[2]).collect()
# for review in reviews:
#     result = NaiveBayes(review, rdd_joined)
    
#     value_real = rdd_result.filter(lambda x: x[2] == review).first()[0]
    
#     classification.append(value_real, result)
    
# word = 'adney'
# try:
#     print(rdd_joined.filter(lambda x: x[0] == word).first())
#     x,y = rdd_joined.filter(lambda x: x[0] == word).first()[1]
#     print(x,y)
# except:
#     print("não tem")
    
# v = rdd_joined.map(lambda x: (x, ''))
# rdd_result.take(10)

# print(classification[0:10])

# reviews.take(5)

column_values = rdd.map(lambda x: x[2])

# Converta o RDD em uma lista usando o método collect()
column_list = column_values.collect()

ConnectionRefusedError: [Errno 111] Connection refused

### Fase 2

Agora que temos um classificador Naive-Bayes, vamos explorá-lo um pouco:

### Tarefa

Quais são as 100 palavras que mais indicam negatividade, ou seja, onde a diferença entre a probabilidade da palavra no conjunto dos comentários negativos e positivos é máxima? E quais as 100 palavras de maior positividade? Mostre os resultados na forma de *word clouds*.

### Tarefa desafio!

Qual o desempenho do classificador (acurácia)? Para medir sua acurácia:

- Separe os reviews em dois conjuntos: treinamente e teste
- Repita o "treinamento" do classificador com o conjunto de treinamento
- Para cada review do conjunto de teste, determine se é positiva ou negativa de acordo com o classificador
- Determine a acurácia

Esta não é uma tarefa trivial. Não basta fazer um `for` para determinar a classe de cada review de teste: isso demoraria uma eternidade. Você tem que usar variáveis "broadcast" do Spark para enviar uma cópia da tabela de frequencias para cada *core* do executor.

### Tarefa desafio!

Implemente Laplace smoothing

## Rubrica de avaliação

- I: groselha, falha crítica, ou não entregou nada
- D: Fez uma tentativa honesta de fazer todos os itens da fase 1, mas tem erros
- C: Fase 1 completa
- B: Fase 2, faltando apenas um desafio
- A: Fase 2 completa